EXTRACT

In [1]:
import pandas as pd

# EXTRACT
df = pd.read_csv("transactions_ip.csv")
df['transaction_date'] = pd.to_datetime(df['transaction_date'], errors='coerce')

# Simpan untuk referensi
df.to_csv("/tmp/extracted_shipping.csv", index=False)
df.head()

,transaction_id,customer_id,transaction_date,total_amount,payment_method,shipping_method,delivery_time,coupon_code,city,product_id,sales_factor
0,1,506,2023-07-22,11300000.0,Cash on Delivery,Tiki,1 minggu,GRATISONGKIR,Jakarta,12,0.7
1,2,23275,2024-07-01,11053000.0,DANA,Tiki,2-3 hari,SALE20,Bandung,3,0.9
2,3,24101,2022-11-01,12741000.0,Transfer Bank,JNE,4-5 hari,NaN,Bandung,5,0.8
3,4,24188,2022-02-04,4921000.0,OVO,JNE,4-5 hari,SALE20,Yogyakarta,10,0.6
4,5,33766,2022-02-06,2163000.0,Transfer Bank,GrabExpress,1 minggu,NaN,Jakarta,11,0.7


TRANSFORM

dim_date

In [2]:
dim_date = df[['transaction_date']].drop_duplicates().copy()
dim_date['date_id'] = dim_date['transaction_date'].dt.strftime('%Y%m%d').astype(int)
dim_date['year'] = dim_date['transaction_date'].dt.year
dim_date['month'] = dim_date['transaction_date'].dt.month
dim_date['day'] = dim_date['transaction_date'].dt.day

dim_city

In [3]:
dim_city = df[['city']].drop_duplicates().copy()
dim_city['city_id'] = dim_city.index + 1

dim_shipping_method

In [4]:
dim_shipping_method = df[['shipping_method']].drop_duplicates().copy()
dim_shipping_method['shipping_method_id'] = dim_shipping_method.index + 1

dim_delivery_category

In [5]:
def categorize_delivery_time(val):
    val = str(val).lower()
    if '1' in val:
        return 'Cepat'
    elif '2-3' in val or '2' in val or '3' in val:
        return 'Sedang'
    elif '4' in val or '5' in val or 'minggu' in val:
        return 'Lambat'
    return 'Tidak Diketahui'

df['delivery_category'] = df['delivery_time'].fillna('Tidak Diketahui').apply(categorize_delivery_time)

dim_delivery_category = df[['delivery_category']].drop_duplicates().copy()
dim_delivery_category['delivery_category_id'] = dim_delivery_category.index + 1


fact_shipping

In [6]:
fact_shipping = df.merge(dim_date, on='transaction_date') \
                  .merge(dim_city, on='city') \
                  .merge(dim_shipping_method, on='shipping_method') \
                  .merge(dim_delivery_category, on='delivery_category')

fact_shipping = fact_shipping[[
    'transaction_id', 'date_id', 'city_id',
    'shipping_method_id', 'delivery_category_id'
]]

LOAD

In [7]:
dim_date.to_csv("/tmp/dim_date_shipping.csv", index=False)
dim_city.to_csv("/tmp/dim_city_shipping.csv", index=False)
dim_shipping_method.to_csv("/tmp/dim_shipping_method.csv", index=False)
dim_delivery_category.to_csv("/tmp/dim_delivery_category.csv", index=False)
fact_shipping.to_csv("/tmp/fact_shipping.csv", index=False)